# Load libraries and data

Enter your project root:

In [1]:
project_root = '/home/dan1dr/zrive-ds-4q24-churn'

In [2]:
import sys
import os
import logging
import pandas as pd
import logging

# Define the project root path
current_wd = os.getcwd()

# Change the working directory if necessary
if current_wd != project_root:
    print(f"Changing working directory from {current_wd} to {project_root}")
    os.chdir(project_root)
else:
    print("Already in the correct path")

# Add 'src' directory to sys.path
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    print(f"Adding {src_path} to sys.path")
    sys.path.insert(0, src_path)

# Import the modules
from db_connectors.bigquery_service import BigqueryService
from data_gathering import data_gathering
from utils.logger import get_logger
logger = get_logger(__name__)

Changing working directory from /home/dan1dr/zrive-ds-4q24-churn/src/eda to /home/dan1dr/zrive-ds-4q24-churn
Adding /home/dan1dr/zrive-ds-4q24-churn/src to sys.path


### Extract the data from BigQuery

Define the query:

In [17]:
query_sql_22 = """SELECT *
FROM `mm-bi-catedras-upm.ESTIMACION_CHURN.multibrand_monthly_customer_base_mp2022`
  LIMIT 1000000
  """

In [18]:
query_sql_23 = """SELECT *
FROM `mm-bi-catedras-upm.ESTIMACION_CHURN.multibrand_monthly_customer_base_mp2022`
  """

In [3]:
#sample = data_gathering(query_sql_22)
logger.info(f"Extraction completed - data_2022")
#data_2023 = data_gathering(query_sql_23)
#logging.info(f"Extraction completed - data_2023")

INFO - Extraction completed - data_2022


### Save the data

In [11]:
save_path = os.path.join(project_root, 'data')
sample.to_parquet(os.path.join(save_path, 'sample_data.parquet'))

### Load the data

In [12]:
read_path = os.path.join(save_path, 'sample_data.parquet')
sample = pd.read_parquet(read_path)

## Explore data

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [14]:
sample.head()

,brand_ds,customer_id,YEAR,MONTH,EOP,IS_CUST_SEGM_RESI,IS_CUST_SEGM_SOHO,IS_CUST_SEGM_BUSI,CHANNEL_LAST_MOBILE,CHANNEL_FIRST_MOBILE,CHANNEL_LAST_FIXE,CHANNEL_FIRST_FIXE,NUM_LINE_TYPE_FIXE_CURR,NUM_LINES_POST_CURR,NUM_LINE_TYPE_SUPD_CURR,NUM_LINES_PREP_CURR,NUM_LINE_TYPE_MAIN_POST_CURR,NUM_LINE_TYPE_ADDI_POST_CURR,NUM_LINE_TYPE_EXTR_POST_CURR,NUM_LINE_TYPE_ADDI_CURR,NUM_LINE_TYPE_EXTR_CURR,NUM_LINE_TYPE_MAIN_CURR,NUM_LINE_TYPE_SUPD_ACTI,NUM_LINES_POST_ACTI,NUM_LINES_PREP_ACTI,NUM_LINE_TYPE_FIXE_POST_ACTI,NUM_LINE_TYPE_MAIN_POST_ACTI,NUM_LINE_TYPE_ADDI_POST_ACTI,NUM_LINE_TYPE_EXTR_POST_ACTI,NUM_LINE_TYPE_SUPD_DEAC,NUM_LINE_TYPE_FIXE_POST_DEAC,NUM_LINES_POST_DEAC,NUM_LINES_PREP_DEAC,NUM_LINE_TYPE_MAIN_POST_DEAC,NUM_LINE_TYPE_EXTR_POST_DEAC,NUM_LINE_TYPE_ADDI_POST_DEAC,NUM_DAYS_LINE_TYPE_FIXE_POST_ACT,NUM_DAYS_LINE_TYPE_MAIN_POST_ACT,NUM_DAYS_LINE_TYPE_ADDI_POST_ACT,NUM_DAYS_LINE_TYPE_EXTR_POST_ACT,NUM_DAYS_LINE_TYPE_MAIN_PREP_ACT,NUM_DAYS_LINE_TYPE_SUPD_ACT,NUM_DAYS_LINE_TYPE_FIXE_POST_DEA,NUM_DAYS_LINE_TYPE_MAIN_POST_DEA,NUM_DAYS_LINE_TYPE_ADDI_POST_DEA,NUM_DAYS_LINE_TYPE_EXTR_POST_DEA,NUM_DAYS_LINE_TYPE_MAIN_PREP_DEA,NUM_DAYS_LINE_TYPE_SUPD_DEA,NUM_LINE_TYPE_FIXE_TECH_FTTH,NUM_LINE_TYPE_FIXE_TECH_ADSL,IS_CUST_CURR,IS_CUST_BILL_POST_CURR,NUM_LINE_TYPE_FIXE_TOTA,NUM_LINE_TYPE_ADDI_POST_TOTA,NUM_LINES_PREP_TOTA,NUM_LINE_TYPE_ADDI_TOTA,NUM_LINE_TYPE_EXTR_TOTA,NUM_LINE_TYPE_MAIN_TOTA,MIN_SERV_ACTIVATION_DATE,LINE_TYPE_FIXE_POST_ACT_LAST_DT,LINE_TYPE_MAIN_POST_ACT_LAST_DT,LINE_TYPE_EXTR_POST_ACT_LAST_DT,LINE_TYPE_ADDI_POST_ACT_LAST_DT,NUM_DAYS_ACT,NUM_DAYS_DEA,NUM_LINES_TOTA,NUM_MONTHS_ACT,NUM_MONTHS_LINE_TYPE_MAIN_POST_ACT,CUST_BUNDLE_CURR,CUST_BUNDLE_TYPE_30D,CUST_BUNDLE_TYPE_120D,NUM_CUSTOMER_BUNDLE_MO,NUM_CUSTOMER_BUNDLE_FMC,NUM_CUSTOMER_BUNDLE_2P,ORDER_FIX_FROM_PORTABILITY,ORDER_FIX_FROM_NEW_ALTA,ORDER_MOBILE_FROM_PORTABILITY,order_mobile_from_new_alta,order_mobile_from_migra_post_to_pre,order_mobile_from_migra_pre_to_post,ORDER_FIX_FROM_PORTABILITY_ACTI,ORDER_FIX_FROM_NEW_ALTA_ACTI,ORDER_MOBILE_FROM_PORTABILITY_ACTI,order_mobile_from_new_alta_ACTI,service_mobile_pending_install,service_fix_pending_install,service_mobile_cancelled,service_fix_cancelled,service_mobile_pending_install_3month,service_fix_pending_install_3month,service_mobile_cancelled_3month,service_fix_cancelled_3month,service_mobile_pending_install_6month,service_fix_pending_install_6month,service_mobile_cancelled_6month,service_fix_cancelled_6month,WEEK,NUM_DISC_ACTI,NUM_DISC_DEAC,NUM_DISC_CURR,NUM_DISC_COMPENSATION_ACTI,NUM_DISC_COMPENSATION_DEAC,NUM_DISC_COMPENSATION_CURR,NUM_DISC_FOREVER_ACTI,NUM_DISC_FOREVER_DEAC,NUM_DISC_FOREVER_CURR,NUM_DISC_PURE_ACTI,NUM_DISC_PURE_DEAC,NUM_DISC_PURE_CURR,NUM_DISC_NEXT_MONTH_ACTI,NUM_DISC_NEXT_MONTH_DEAC,NUM_DISC_NEXT_2_MONTHS_ACTI,NUM_DISC_NEXT_2_MONTHS_DEAC,NUM_DISC_PURE_NEXT_MONTH_ACTI,NUM_DISC_PURE_NEXT_MONTH_DEAC,NUM_DISC_PURE_NEXT_2_MONTHS_ACTI,NUM_DISC_PURE_NEXT_2_MONTHS_DEAC,NUM_EURO_DISC_CURR,MAX_DEACTIVATION_DATE,MIN_DEACTIVATION_DATE,MAX_ACTIVATION_DATE,MIN_ACTIVATION_DATE,MAX_PURE_DISC_DEACTIVATION_DATE,MIN_PURE_DISC_DEACTIVATION_DATE,MAX_PURE_DISC_ACTIVATION_DATE,MIN_PURE_DISC_ACTIVATION_DATE,NUM_DAYS_MAX_DEACTIVATION_DATE,NUM_MONTHS_MAX_DEACTIVATION_DATE,NUM_DAYS_MAX_PURE_DISC_DEACTIVATION_DATE,NUM_MONTHS_MAX_PURE_DISC_DEACTIVATION_DATE,NUM_MONTHS_MAX_DISC_PURE_DEACTIVATION_DATE,MAX_DISC_DEACTIVATION_DATE,MAX_DISC_PURE_DEACTIVATION_DATE,MAX_DISC_DEACTIVATION_DATE_D,LINE_TYPE_FIXE_POST_ACT_LAST_DT_D,NUM_PERM_CURR,MAX_DAYS_PERM_CURR,MIN_DAYS_PERM_CURR,MIN_PENALTY_AMOUNT_CURR,MAX_PENALTY_AMOUNT_CURR,NUM_PREV_OBJ,NUM_DUR_OBJ,MIN_ACTIVE_DATE,PREV_FINISHED_PERM,NUM_PERM_DEAC,NUM_PERM_ANTI_DEAC,pago_final_0,dif_pago_final_prev_month,dif_pago_final_prev_2_month,dif_pago_final_prev_3_month,periodica_0,dif_periodica_prev_month,dif_periodica_prev_2_month,dif_periodica_prev_3_month,consumo_0,dif_consumo_prev_month,dif_consumo_prev_2_month,dif_consumo_prev_3_month,aperiodica_0,dif_aperiodica_prev_month,dif_aperiodica_prev

In [15]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 430 entries, brand_ds to NUM_PORT_OPER_RECE_YOIGO_TWO_YEAR_AGO
dtypes: Int64(270), boolean(2), datetime64[us, UTC](1), float64(125), object(32)
memory usage: 3.4+ GB


In [16]:
sample.columns.tolist()

['brand_ds',
 'customer_id',
 'YEAR',
 'MONTH',
 'EOP',
 'IS_CUST_SEGM_RESI',
 'IS_CUST_SEGM_SOHO',
 'IS_CUST_SEGM_BUSI',
 'CHANNEL_LAST_MOBILE',
 'CHANNEL_FIRST_MOBILE',
 'CHANNEL_LAST_FIXE',
 'CHANNEL_FIRST_FIXE',
 'NUM_LINE_TYPE_FIXE_CURR',
 'NUM_LINES_POST_CURR',
 'NUM_LINE_TYPE_SUPD_CURR',
 'NUM_LINES_PREP_CURR',
 'NUM_LINE_TYPE_MAIN_POST_CURR',
 'NUM_LINE_TYPE_ADDI_POST_CURR',
 'NUM_LINE_TYPE_EXTR_POST_CURR',
 'NUM_LINE_TYPE_ADDI_CURR',
 'NUM_LINE_TYPE_EXTR_CURR',
 'NUM_LINE_TYPE_MAIN_CURR',
 'NUM_LINE_TYPE_SUPD_ACTI',
 'NUM_LINES_POST_ACTI',
 'NUM_LINES_PREP_ACTI',
 'NUM_LINE_TYPE_FIXE_POST_ACTI',
 'NUM_LINE_TYPE_MAIN_POST_ACTI',
 'NUM_LINE_TYPE_ADDI_POST_ACTI',
 'NUM_LINE_TYPE_EXTR_POST_ACTI',
 'NUM_LINE_TYPE_SUPD_DEAC',
 'NUM_LINE_TYPE_FIXE_POST_DEAC',
 'NUM_LINES_POST_DEAC',
 'NUM_LINES_PREP_DEAC',
 'NUM_LINE_TYPE_MAIN_POST_DEAC',
 'NUM_LINE_TYPE_EXTR_POST_DEAC',
 'NUM_LINE_TYPE_ADDI_POST_DEAC',
 'NUM_DAYS_LINE_TYPE_FIXE_POST_ACT',
 'NUM_DAYS_LINE_TYPE_MAIN_POST_ACT',
 'NUM_D

In [17]:
def assess_NA(data: pd.DataFrame):
    """
    Returns a pd.DataFrame denoting the total number of NA
    values and the percentage of NA values in each column.
    """
    # pd.Datadenoting features and the sum of their null values
    nulls = data.isnull().sum().reset_index().rename(columns={0: "count"})
    nulls["percent"] = nulls["count"] * 100 / len(data)

    return nulls

nulls = assess_NA(sample)


In [18]:
nulls.head(20)

,index,count,percent
0,brand_ds,0,0.0000
1,customer_id,0,0.0000
2,YEAR,0,0.0000
3,MONTH,0,0.0000
4,EOP,0,0.0000
5,IS_CUST_SEGM_RESI,0,0.0000
6,IS_CUST_SEGM_SOHO,0,0.0000
7,IS_CUST_SEGM_BUSI,0,0.0000
8,CHANNEL_LAST_MOBILE,71713,7.1713
9,CHANNEL_FIRST_MOBILE,59364,5.9364


In [19]:
na_info_sorted = nulls.sort_values(by='percent', ascending=False)
na_info_sorted.head(20)


,index,count,percent
45,NUM_DAYS_LINE_TYPE_EXTR_POST_DEA,1000000,100.0000
61,LINE_TYPE_EXTR_POST_ACT_LAST_DT,1000000,100.0000
175,Month_purchase_modem,1000000,100.0000
176,Month_purchase_desc,1000000,100.0000
39,NUM_DAYS_LINE_TYPE_EXTR_POST_ACT,1000000,100.0000
47,NUM_DAYS_LINE_TYPE_SUPD_DEA,999979,99.9979
41,NUM_DAYS_LINE_TYPE_SUPD_ACT,999760,99.9760
421,MIN_NUM_DAYS_BETWEEN_FIX_PORT,999703,99.9703
420,AVG_NUM_DAYS_BETWEEN_FIX_PORT,999703,99.9703
417,MIN_NUM_DAYS_BETWEEN_FIX_PORT_REQS,999072,99.9072


In [23]:
sample['customer_id'].nunique()

250830

In [114]:
# Select rows where 'IS_CUST_CURR' is null
churned = sample[pd.isnull(sample['IS_CUST_CURR'])]

In [21]:
sample['IS_CUST_CURR'].value_counts()

IS_CUST_CURR
True    1000000
Name: count, dtype: Int64

In [27]:
sample[sample['customer_id'] == '7270556']

,brand_ds,customer_id,YEAR,MONTH,EOP,IS_CUST_SEGM_RESI,IS_CUST_SEGM_SOHO,IS_CUST_SEGM_BUSI,CHANNEL_LAST_MOBILE,CHANNEL_FIRST_MOBILE,CHANNEL_LAST_FIXE,CHANNEL_FIRST_FIXE,NUM_LINE_TYPE_FIXE_CURR,NUM_LINES_POST_CURR,NUM_LINE_TYPE_SUPD_CURR,NUM_LINES_PREP_CURR,NUM_LINE_TYPE_MAIN_POST_CURR,NUM_LINE_TYPE_ADDI_POST_CURR,NUM_LINE_TYPE_EXTR_POST_CURR,NUM_LINE_TYPE_ADDI_CURR,NUM_LINE_TYPE_EXTR_CURR,NUM_LINE_TYPE_MAIN_CURR,NUM_LINE_TYPE_SUPD_ACTI,NUM_LINES_POST_ACTI,NUM_LINES_PREP_ACTI,NUM_LINE_TYPE_FIXE_POST_ACTI,NUM_LINE_TYPE_MAIN_POST_ACTI,NUM_LINE_TYPE_ADDI_POST_ACTI,NUM_LINE_TYPE_EXTR_POST_ACTI,NUM_LINE_TYPE_SUPD_DEAC,NUM_LINE_TYPE_FIXE_POST_DEAC,NUM_LINES_POST_DEAC,NUM_LINES_PREP_DEAC,NUM_LINE_TYPE_MAIN_POST_DEAC,NUM_LINE_TYPE_EXTR_POST_DEAC,NUM_LINE_TYPE_ADDI_POST_DEAC,NUM_DAYS_LINE_TYPE_FIXE_POST_ACT,NUM_DAYS_LINE_TYPE_MAIN_POST_ACT,NUM_DAYS_LINE_TYPE_ADDI_POST_ACT,NUM_DAYS_LINE_TYPE_EXTR_POST_ACT,NUM_DAYS_LINE_TYPE_MAIN_PREP_ACT,NUM_DAYS_LINE_TYPE_SUPD_ACT,NUM_DAYS_LINE_TYPE_FIXE_POST_DEA,NUM_DAYS_LINE_TYPE_MAIN_POST_DEA,NUM_DAYS_LINE_TYPE_ADDI_POST_DEA,NUM_DAYS_LINE_TYPE_EXTR_POST_DEA,NUM_DAYS_LINE_TYPE_MAIN_PREP_DEA,NUM_DAYS_LINE_TYPE_SUPD_DEA,NUM_LINE_TYPE_FIXE_TECH_FTTH,NUM_LINE_TYPE_FIXE_TECH_ADSL,IS_CUST_CURR,IS_CUST_BILL_POST_CURR,NUM_LINE_TYPE_FIXE_TOTA,NUM_LINE_TYPE_ADDI_POST_TOTA,NUM_LINES_PREP_TOTA,NUM_LINE_TYPE_ADDI_TOTA,NUM_LINE_TYPE_EXTR_TOTA,NUM_LINE_TYPE_MAIN_TOTA,MIN_SERV_ACTIVATION_DATE,LINE_TYPE_FIXE_POST_ACT_LAST_DT,LINE_TYPE_MAIN_POST_ACT_LAST_DT,LINE_TYPE_EXTR_POST_ACT_LAST_DT,LINE_TYPE_ADDI_POST_ACT_LAST_DT,NUM_DAYS_ACT,NUM_DAYS_DEA,NUM_LINES_TOTA,NUM_MONTHS_ACT,NUM_MONTHS_LINE_TYPE_MAIN_POST_ACT,CUST_BUNDLE_CURR,CUST_BUNDLE_TYPE_30D,CUST_BUNDLE_TYPE_120D,NUM_CUSTOMER_BUNDLE_MO,NUM_CUSTOMER_BUNDLE_FMC,NUM_CUSTOMER_BUNDLE_2P,ORDER_FIX_FROM_PORTABILITY,ORDER_FIX_FROM_NEW_ALTA,ORDER_MOBILE_FROM_PORTABILITY,order_mobile_from_new_alta,order_mobile_from_migra_post_to_pre,order_mobile_from_migra_pre_to_post,ORDER_FIX_FROM_PORTABILITY_ACTI,ORDER_FIX_FROM_NEW_ALTA_ACTI,ORDER_MOBILE_FROM_PORTABILITY_ACTI,order_mobile_from_new_alta_ACTI,service_mobile_pending_install,service_fix_pending_install,service_mobile_cancelled,service_fix_cancelled,service_mobile_pending_install_3month,service_fix_pending_install_3month,service_mobile_cancelled_3month,service_fix_cancelled_3month,service_mobile_pending_install_6month,service_fix_pending_install_6month,service_mobile_cancelled_6month,service_fix_cancelled_6month,WEEK,NUM_DISC_ACTI,NUM_DISC_DEAC,NUM_DISC_CURR,NUM_DISC_COMPENSATION_ACTI,NUM_DISC_COMPENSATION_DEAC,NUM_DISC_COMPENSATION_CURR,NUM_DISC_FOREVER_ACTI,NUM_DISC_FOREVER_DEAC,NUM_DISC_FOREVER_CURR,NUM_DISC_PURE_ACTI,NUM_DISC_PURE_DEAC,NUM_DISC_PURE_CURR,NUM_DISC_NEXT_MONTH_ACTI,NUM_DISC_NEXT_MONTH_DEAC,NUM_DISC_NEXT_2_MONTHS_ACTI,NUM_DISC_NEXT_2_MONTHS_DEAC,NUM_DISC_PURE_NEXT_MONTH_ACTI,NUM_DISC_PURE_NEXT_MONTH_DEAC,NUM_DISC_PURE_NEXT_2_MONTHS_ACTI,NUM_DISC_PURE_NEXT_2_MONTHS_DEAC,NUM_EURO_DISC_CURR,MAX_DEACTIVATION_DATE,MIN_DEACTIVATION_DATE,MAX_ACTIVATION_DATE,MIN_ACTIVATION_DATE,MAX_PURE_DISC_DEACTIVATION_DATE,MIN_PURE_DISC_DEACTIVATION_DATE,MAX_PURE_DISC_ACTIVATION_DATE,MIN_PURE_DISC_ACTIVATION_DATE,NUM_DAYS_MAX_DEACTIVATION_DATE,NUM_MONTHS_MAX_DEACTIVATION_DATE,NUM_DAYS_MAX_PURE_DISC_DEACTIVATION_DATE,NUM_MONTHS_MAX_PURE_DISC_DEACTIVATION_DATE,NUM_MONTHS_MAX_DISC_PURE_DEACTIVATION_DATE,MAX_DISC_DEACTIVATION_DATE,MAX_DISC_PURE_DEACTIVATION_DATE,MAX_DISC_DEACTIVATION_DATE_D,LINE_TYPE_FIXE_POST_ACT_LAST_DT_D,NUM_PERM_CURR,MAX_DAYS_PERM_CURR,MIN_DAYS_PERM_CURR,MIN_PENALTY_AMOUNT_CURR,MAX_PENALTY_AMOUNT_CURR,NUM_PREV_OBJ,NUM_DUR_OBJ,MIN_ACTIVE_DATE,PREV_FINISHED_PERM,NUM_PERM_DEAC,NUM_PERM_ANTI_DEAC,pago_final_0,dif_pago_final_prev_month,dif_pago_final_prev_2_month,dif_pago_final_prev_3_month,periodica_0,dif_periodica_prev_month,dif_periodica_prev_2_month,dif_periodica_prev_3_month,consumo_0,dif_consumo_prev_month,dif_consumo_prev_2_month,dif_consumo_prev_3_month,aperiodica_0,dif_aperiodica_prev_month,dif_aperiodica_prev

In [31]:
sample.dtypes.tolist()

[dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 BooleanDtype,
 BooleanDtype,
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 Int64Dtype(),
 Int64Dtype(),
 Int64Dtype(),
 dtype('float64'),
 dtype('float64'),
